In [1]:
import random
from hashlib import sha256


def coprime(a, b):
    while b != 0:
        a, b = b, a % b
    return a
    
    
def extended_gcd(aa, bb):
    lastremainder, remainder = abs(aa), abs(bb)
    x, lastx, y, lasty = 0, 1, 1, 0
    while remainder:
        lastremainder, (quotient, remainder) = remainder, divmod(lastremainder, remainder)
        x, lastx = lastx - quotient*x, x
        y, lasty = lasty - quotient*y, y
    return lastremainder, lastx * (-1 if aa < 0 else 1), lasty * (-1 if bb < 0 else 1)

#Euclid's extended algorithm for finding the multiplicative inverse of two numbers    
def modinv(a, m):
	g, x, y = extended_gcd(a, m)
	if g != 1:
		raise Exception('Modular inverse does not exist')
	return x % m    

        
def is_prime(num):
    if num == 2:
        return True
    if num < 2 or num % 2 == 0:
        return False
    for n in range(3, int(num**0.5)+2, 2):
        if num % n == 0:
            return False
    return True


def generate_keypair(p, q):
    if not (is_prime(p) and is_prime(q)):
        raise ValueError('Both numbers must be prime.')
    elif p == q:
        raise ValueError('p and q cannot be equal')

    n = p * q

    #Phi is the totient of n
    phi = (p-1) * (q-1)

    #Choose an integer e such that e and phi(n) are coprime
    e = random.randrange(1, phi)

    #Use Euclid's Algorithm to verify that e and phi(n) are comprime 
    g = coprime(e, phi)
  
    while g != 1:
        e = random.randrange(1, phi)
        g = coprime(e, phi)

    #Use Extended Euclid's Algorithm to generate the private key
    d = modinv(e, phi)

    #Return public and private keypair
    #Public key is (e, n) and private key is (d, n)
    return ((e, n), (d, n))


def encrypt(privatek, plaintext):
    #Unpack the key into it's components
    key, n = privatek

    #Convert each letter in the plaintext to numbers based on the character using a^b mod m
            
    numberRepr = [ord(char) for char in plaintext]
    print("Number representation before encryption: ", numberRepr)
    cipher = [pow(ord(char),key,n) for char in plaintext]
    
    #Return the array of bytes
    return cipher


def decrypt(publick, ciphertext):
    #Unpack the key into its components
    key, n = publick
       
    #Generate the plaintext based on the ciphertext and key using a^b mod m
    numberRepr = [pow(char, key, n) for char in ciphertext]
    plain = [chr(pow(char, key, n)) for char in ciphertext]

    print("Decrypted number representation is: ", numberRepr)
    
    #Return the array of bytes as a string
    return ''.join(plain)
    
    
def hashFunction(message):
    hashed = sha256(message.encode("UTF-8")).hexdigest()
    return hashed
    
    
def verify(receivedHashed, message):
    ourHashed = hashFunction(message)
    if receivedHashed == ourHashed:
        print("Verification successful: ", )
        print(receivedHashed, " = ", ourHashed)
    else:
        
        print("Verification failed")
        print(receivedHashed, " != ", ourHashed)
        

def main():
    p = int(input("Enter a prime number (17, 19, 23, etc): "))
    q = int(input("Enter another prime number (Not one you entered above): "))   
    #p = 17
    #q=23
    
    
    print("Generating your public/private keypairs now . . .")
    public, private = generate_keypair(p, q)
    
    print("Your public key is ", public ," and your private key is ", private)
    message = input("Enter a message to encrypt with your private key: ")
    print("")

    hashed = hashFunction(message)
    
    print("Encrypting message with private key ", private ," . . .")
    encrypted_msg = encrypt(private, hashed)   
    print("Your encrypted hashed message is: ")
    print(''.join(map(lambda x: str(x), encrypted_msg)))
    #print(encrypted_msg)
    
    print("")
    print("Decrypting message with public key ", public ," . . .")

    decrypted_msg = decrypt(public, encrypted_msg)
    print("Your decrypted message is:")  
    print(decrypted_msg)
    
    print("")
    print("Verification process . . .")
    verify(decrypted_msg, message)
   
main()    
    
    
    
    
    
    
    
    
    

Enter a prime number (17, 19, 23, etc): 17
Enter another prime number (Not one you entered above): 61
Generating your public/private keypairs now . . .
Your public key is  (239, 1037)  and your private key is  (719, 1037)
Enter a message to encrypt with your private key: my name is rishabha

Encrypting message with private key  (719, 1037)  . . .
Number representation before encryption:  [56, 50, 97, 98, 52, 98, 48, 48, 55, 97, 51, 50, 49, 101, 50, 52, 49, 99, 50, 55, 53, 53, 101, 102, 51, 98, 97, 54, 48, 57, 98, 55, 100, 100, 48, 49, 102, 98, 49, 101, 57, 54, 56, 50, 48, 100, 101, 49, 97, 55, 102, 98, 48, 50, 53, 97, 98, 101, 57, 99, 50, 57, 53, 97]
Your encrypted hashed message is: 
500560588582698582929929132588799560127883560698127419560132587587883918799582588941929564582132280280929127918582127883564941500560929280883127588132918582929560587588582883564419560564587588

Decrypting message with public key  (239, 1037)  . . .
Decrypted number representation is:  [56, 50, 97, 98, 52,